In [134]:
# import necessary libraries
import pandas as pd
import re
from airportsdata import load
from actions.done.CreateZFWMessageAction import CreateZFWMessageAction
from actions.done.UpdateEstimatesAction import UpdateEstimatesAction
from actions.done.CalculateWeightAndTrimAction import CalculateWeightAndTrimAction
from actions.done.CreateLoadsheetAction import CreateLoadsheetAction
import json
import os

def load_config(path):
    with open(path, 'r') as file:
        return json.load(file)

config_path = '../pandas_config.json'
config = load_config(config_path)
parquet_config = config['parquet']

# Setting up the engine as a global constant
ENGINE = parquet_config['engine']
COMPRESSION = parquet_config['compression']
FILE_PATH = "../data"

In [135]:
df = pd.read_parquet(f"{FILE_PATH}/data_parquet/processed_data_combined.parquet", engine=ENGINE)

In [145]:
os.listdir(f"actions/done")
ACTIONS = [action.replace(".py", "") for action in os.listdir(f"actions/done") if action.endswith(".py")]
df = df[df['action_name'].isin(ACTIONS)]

In [137]:
flight_ids = df['flight_id'].unique()

In [139]:
estimated_zfws = {}
actual_zfws = {}
actual_take_off_fuels = {}
actual_tows = {}
for flight in flight_ids[:100]:
    subset_df = df[df['flight_id'] == flight]
    subset_df.sort_index(inplace=True)
    actions = subset_df['action_name'].values
    entriy_details = subset_df["entry_details"].values
    temp_estimated_zfws = []
    temp_actual_zfws = []
    temp_actual_fuels = []
    temp_actual_tows = []
    for entry, action in zip(entriy_details, actions):
        if action == "CreateZFWMessageAction":
            estimated_zfw, actual_zfw = CreateZFWMessageAction(entry)
            if estimated_zfw != None:
                temp_estimated_zfws.append(estimated_zfw)
            if actual_zfw != None:
                temp_actual_zfws.append(actual_zfw)
        if action == "UpdateEstimatesAction":
            estimated_zfw = UpdateEstimatesAction(entry)
            if estimated_zfw != None:
                temp_estimated_zfws.append(estimated_zfw)
            if actual_zfw != None:
                temp_actual_zfws.append(actual_zfw)

        if action == "CalculateWeightAndTrimAction":
            actual_zfw = CalculateWeightAndTrimAction(entry)
            if actual_zfw != None:
                temp_actual_zfws.append(actual_zfw)
        if action == "CreateLoadsheetAction":
            actual_tow, actual_zfw, estimated_zfw, actual_fuel = CreateLoadsheetAction(entry)
            if actual_zfw != None:
                temp_actual_zfws.append(actual_zfw)
            if estimated_zfw != None:
                temp_estimated_zfws.append(estimated_zfw)
            if actual_fuel != None:
                temp_actual_fuels.append(actual_fuel)
            if actual_tow != None:
                temp_actual_tows.append(actual_tow)
    estimated_zfws[flight] = temp_estimated_zfws
    actual_zfws[flight] = temp_actual_zfws
    actual_take_off_fuels[flight] = temp_actual_fuels
    actual_tows[flight] = temp_actual_tows

In [140]:
estimated_zfws_tuples = [(k, v) for k, v in estimated_zfws.items()]
actual_zfws_tuples = [(k, v) for k, v in actual_zfws.items()]

# Create DataFrame
estimated_zfws_df = pd.DataFrame(estimated_zfws_tuples, columns=['flight_number', 'estimated_zfws'])
actual_zfws_df = pd.DataFrame(actual_zfws_tuples, columns=['flight_number', 'actual_zfws'])
weights = pd.merge(estimated_zfws_df, actual_zfws_df, on='flight_number', how="outer")


In [7]:
weights.to_parquet("../data/data_parquet/weights.parquet", engine=ENGINE, compression=COMPRESSION)